In [1]:
import pandas as pd
import numpy as np

## Data Integration
- Actors and Directors Who Cooperated At Least Three Times


In [2]:
actors_and_directors = pd.DataFrame({
    'actor_id': [1, 1, 1, 1, 1, 2, 2],
    'director_id': [1, 1, 1, 2, 2, 1, 1],
    'timestamp': [0, 1, 2, 3, 4, 5, 6]
})
actors_and_directors

,actor_id,director_id,timestamp
0,1,1,0
1,1,1,1
2,1,1,2
3,1,2,3
4,1,2,4
5,2,1,5
6,2,1,6


In [3]:
actors_and_directors1 = actors_and_directors.groupby(["actor_id","director_id"])["timestamp"].count().reset_index()

In [4]:
actors_and_directors1

,actor_id,director_id,timestamp
0,1,1,3
1,1,2,2
2,2,1,2


In [5]:
actors_and_directors1.loc[actors_and_directors1.timestamp >= 3,["actor_id","director_id"]]

,actor_id,director_id
0,1,1
